In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# This is sentiment classifier in amazon reviews on mobiles 

In [16]:
data = pd.read_csv("/kaggle/input/amazon-reviews-unlocked-mobile-phones/Amazon_Unlocked_Mobile.csv",)

In [19]:
data.sample(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
345376,Samsung Galaxy S4 Mini L520 16GB Sprint CDMA 4...,Samsung,89.99,5,I LIKED IT SO MUCH I BOUGHT A NEW ONE THIS COM...,0.0
248468,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,it works very well,0.0
194533,HTC One M8 - Factory Unlocked 32GB - US Warran...,HTC,138.99,1,It was missing parts and wouldn't turn on. I h...,0.0
244833,Moto G Plus (4th Gen.) Unlocked - White - 16GB...,Motorola,249.00,5,"Honestly, I was a bit sketchy at first because...",1.0
60584,"Apple iPhone 6 Plus Unlocked Cellphone, 16GB, ...",NaN,490.00,1,I'm not happy because until now i have problem...,2.0
365249,Samsung Galaxy S6 Factory Unlocked 32GB Smartp...,Samsung,414.88,3,automatically restart.,0.0
93016,Blackberry Curve 8520 Unlocked Quad-Band GSM P...,BlackBerry,39.99,5,excelente,0.0
63323,"Apple iPhone 6, Silver, 128 GB (Verizon)",Apple,399.00,5,got this for the wife. so far she likes it.,0.0
219493,LG G3 D855 16GB Metallic Black Factory Unlocke...,LG,275.00,5,Just great.,0.0
89950,Blackberry Bold Touch 9930 CDMA GSM Unlocked P...,BlackBerry,104.95,4,exelent,0.0


In [20]:
data["Reviews"][0]

"I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!"

In [21]:
len(data)

413840

In [24]:
data.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [23]:
data.dropna(inplace=True)

In [28]:
# now make a sentiment col
data["sentiment"] = data["Rating"].apply(lambda rating:+1 if rating > 3 else -1)

In [30]:
data.sample(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,sentiment
230424,"LG Nexus 5X Unlocked Smart Phone, 5.2"" Quartz ...",LG Electronics,399.00,2,Although I love how this phone functions (fast...,1.0,-1
21895,"Apple iPhone 5 Unlocked Cellphone, 32GB, White",Apple,265.00,5,very good iting thanks,0.0,1
23182,"Apple iPhone 5 Unlocked Cellphone, 64GB, White",Apple,314.95,5,Arrived in good shape and works wonderfully.,0.0,1
405998,Verizon HTC Rezound 4G Android Smarphone - 8MP...,HTC,74.99,1,Buyer Beware! The phone that you will get WILL...,0.0,-1
65572,Apple iPhone 6s 16 GB International Warranty U...,Apple,564.95,5,It's very good.,1.0,1
101703,BlackBerry Torch 9810 Unlocked GSM Touchscreen...,BlackBerry,83.73,4,"Muy satisfecha con el producto recibido, cumpl...",0.0,1
189272,HTC DROID INCREDIBLE ADR 6300 3G Android Phone...,HTC,29.99,1,When I ordered the phone it was supposed to be...,0.0,-1
189089,"HTC DROID INCREDIBLE 2 Android Phone, Black (V...",HTC,83.98,3,I bought this phone to replace my Droid X2. I ...,8.0,-1
2232,ALCATEL OneTouch Idol 3 Global Unlocked 4G LTE...,Alcatel,129.00,5,I ordered this cellphone as a gift for a famil...,1.0,1
231984,LG Nexus 5X Unlocked Smartphone - White 32GB (...,LG,399.00,5,Great phone. I like googles case too.,0.0,1


In [31]:
# now preprocess the Reviews col
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(r'[ \n]+', ' ', text)
    return text.strip().lower() 

In [33]:
data["Reviews"] = data["Reviews"].apply(preprocess)

In [34]:
data["Reviews"][0]

"i feel so lucky to have found this used phone to us not used hard at all phone on line from someone who upgraded and sold this one my son liked his old one that finally fell apart after 2 5 years and didn't want an upgrade thank you seller we really appreciate it your honesty re said used phone i recommend this seller very highly would but from them again"

In [36]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.Reviews,data.sentiment,
                        test_size=0.2,random_state=2022,stratify=data.sentiment)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [37]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (267462,)
Shape of X_test:  (66866,)


In [39]:
# using multinomial nb
# now creating the pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('MNB', MultinomialNB())         
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.89      0.72      0.80     20732
           1       0.89      0.96      0.92     46134

    accuracy                           0.89     66866
   macro avg       0.89      0.84      0.86     66866
weighted avg       0.89      0.89      0.88     66866



In [41]:
X_test[:5]

1691      the rubber inner sleeve does not look good on ...
4721                                          good unlocked
73576     i am not satisfied at all i bought i phone shi...
150199    its good i bought it for my child and he absol...
111865                                            muy bueno
Name: Reviews, dtype: object

In [42]:
y_pred[:5]

array([-1,  1, -1,  1,  1])

In [43]:
y_test[:5]

1691     -1
4721      1
73576    -1
150199    1
111865    1
Name: sentiment, dtype: int64